<a href="https://colab.research.google.com/github/gylab-TAU/layer_representation_extraction_service/blob/master/representation_extraction_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone "https://github.com/gylab-TAU/layer_representation_extraction_service.git"
!pip install -r layer_representation_extraction_service/requirements.txt

Cloning into 'layer_representation_extraction_service'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 97 (delta 51), reused 73 (delta 32), pack-reused 0
Receiving objects: 100% (97/97), 15.71 KiB | 5.24 MiB/s, done.
Resolving deltas: 100% (51/51), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.1 MB/s eta 0:00:0

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from layer_representation_extraction_service.models import *
from layer_representation_extraction_service.rdm_calculations import MemoryEfficientRDMCalculator
import ipywidgets as widgets
from pathlib import Path

# create a dropdown widget
dropdown = widgets.Dropdown(
    options=['VGG16 - ImageNet', 'OpenCLIP ViT-B/32'],
    value='VGG16 - ImageNet',
    description='Select a model:',
    disabled=False,
)
display(dropdown)

# create a text input widget
text_input = widgets.Text(
    value='',
    placeholder='/',
    description='Choose images directory:',
    disabled=False
)
display(text_input)

# Input for batch size
batch_size_input = widgets.BoundedIntText(
    value=1,
    min=1,
    max=300,
    step=1,
    description='Batch size:',
    disabled=False
)
display(batch_size_input)


# create a start button
start_button = widgets.Button(description='Create RDMs')

# define a function to be called when the start button is clicked
def start_button_clicked(b):
  if dropdown.value == 'VGG16 - ImageNet':
    model, preprocess, layer_names = get_vgg16_imagenet_resources()
  elif dropdown.value == 'OpenCLIP ViT-B/32':
    model, preprocess, layer_names = get_clip_vit32_resources()

  rdm_calc = MemoryEfficientRDMCalculator(batch_size_input.value)

  imgs_dir = Path(text_input.value)
  if not imgs_dir.is_dir():
    print("Path does not lead to a directory. Please must choose a directory \
            containing images and try again.")
  else:
    outputs = rdm_calc.calc_rdm(model, preprocess, list(imgs_dir.iterdir()), layer_names)
    print(outputs.keys())


# attach the function to the start button
start_button.on_click(start_button_clicked)

display(start_button)


/usr/local/lib/python3.10/dist-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/usr/local/lib/python3.10/dist-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/usr/local/lib/python3.10/dist-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/usr/local/lib/python3.10/dist-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,


Dropdown(description='Select a model:', options=('VGG16 - ImageNet', 'OpenCLIP ViT-B/32'), value='VGG16 - Imag…

Text(value='', description='Choose images directory:', placeholder='/')

BoundedIntText(value=1, description='Batch size:', max=300, min=1)

Button(description='Create RDMs', style=ButtonStyle())

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:11<00:00, 46.2MB/s]


n_batches = 118 / 1 = 118.0
n_iters = 118.0 ^ 2 = 13924.0


  6%|▌         | 7/118 [01:04<16:25,  8.88s/it]

In [1]:
from layer_representation_extraction_service.models import *
from layer_representation_extraction_service.rdm_calculations import MemoryEfficientRDMCalculator
import ipywidgets as widgets
from pathlib import Path

model, preprocess, layer_names = get_vgg16_imagenet_resources()
rdm_calc = MemoryEfficientRDMCalculator(118)
imgs_dir = Path("/content/drive/MyDrive/gylab/Semantic influences on perception/Aude Oliva's data/experiment 2 data/images")
outputs = rdm_calc.calc_rdm(model, preprocess, list(imgs_dir.iterdir()), layer_names)
print(outputs.keys())

/usr/local/lib/python3.10/dist-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/usr/local/lib/python3.10/dist-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/usr/local/lib/python3.10/dist-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/usr/local/lib/python3.10/dist-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,
  0%|          | 0/1 [01:12<?, ?it/s]


KeyError: ignored